# This notebook reproduces all the examples in Sven Vergoolaege's paper "Counting Affine Calculator and Applications."

In [42]:
%pylab inline
# Load islpy and islplot
from islpy import *
from islplot.plotter import *

Populating the interactive namespace from numpy and matplotlib


In [43]:
# Counting Affine Calculator and Applications
# Figure 1:
#    for (i = 0; i < N; ++i)
# S1:  t[i] = f(a[i])
#    for (i = 0; i < N; ++i)
# S2:  b[i] = g(t[N-i-1])
#
# Section 2. Syntax
idom = UnionSet("[n] -> {S1[i] : 0 <= i <= n; S2[i]: 0 <= i <= n};")
print("Iteration Domain: ",type(idom), idom)

Iteration Domain:  <class 'islpy._isl.UnionSet'> [n] -> { S1[i] : 0 <= i <= n; S2[i] : 0 <= i <= n }


In [44]:
read_acc = UnionMap("[n]-> {S1[i] -> a[i]; S2[i] -> t[n-i-1]}")
print("Read access relations: ", type(read_acc), read_acc)

Read access relations:  <class 'islpy._isl.UnionMap'> [n] -> { S1[i] -> a[i]; S2[i] -> t[-1 + n - i] }


In [45]:
write_acc = UnionMap("{S1[i] -> t[i]; S2[i]->b[i]}")
print("Writer access relations: ", type(write_acc), write_acc)

Writer access relations:  <class 'islpy._isl.UnionMap'> { S2[i] -> b[i]; S1[i] -> t[i] }


In [46]:
W = write_acc.intersect_domain(idom)
print(type(W), W)

<class 'islpy._isl.UnionMap'> [n] -> { S2[i] -> b[i] : 0 <= i <= n; S1[i] -> t[i] : 0 <= i <= n }


In [47]:
R = read_acc.intersect_domain(idom)
print(type(R), R)

<class 'islpy._isl.UnionMap'> [n] -> { S2[i] -> t[-1 + n - i] : 0 <= i <= n; S1[i] -> a[i] : 0 <= i <= n }


In [48]:
rmR = R.range_map() # range_map R
print(type(rmR), rmR)

<class 'islpy._isl.UnionMap'> [n] -> { [S2[i] -> t[-1 + n - i]] -> t[-1 + n - i] : 0 <= i <= n; [S1[i] -> a[i]] -> a[i] : 0 <= i <= n }


In [56]:
# Section 3. BASIC OPERATIONS
S = UnionMap("{S1[i]->[0,i];S2[i]->[1,i]}") #schedule.
R.compute_flow(W, W, S)

(0,
 UnionMap("[n] -> { S1[i] -> S2[i' = -1 + n - i] : 0 <= i < n }"),
 UnionMap("[n] -> {  }"),
 UnionMap("[n] -> { S2[i = n] -> t[-1] : n >= 0; S1[i] -> a[i] : 0 <= i <= n }"),
 UnionMap("[n] -> {  }"))

In [57]:
T = R.compute_flow(W,W,S)
DR = T[1] # dependence relation 
print(type(DR),DR)

<class 'islpy._isl.UnionMap'> [n] -> { S1[i] -> S2[i' = -1 + n - i] : 0 <= i < n }


In [58]:
# deltas(S^-1.Dep.S);
invS = S.reverse()
print("Schedule: ", S)
print("S^-1 = ", invS)
print("Dependence Relation:", DR)
DepDist = invS.apply_range(DR).apply_range(S)
print(type(DepDist), DepDist)
DepDist.deltas()

Schedule:  { S2[i] -> [1, i]; S1[i] -> [0, i] }
S^-1 =  { [1, i1] -> S2[i = i1]; [0, i1] -> S1[i = i1] }
Dependence Relation: [n] -> { S1[i] -> S2[i' = -1 + n - i] : 0 <= i < n }
<class 'islpy._isl.UnionMap'> [n] -> { [0, i1] -> [1, -1 + n - i1] : 0 <= i1 < n }


UnionSet("[n] -> { [1, i1] : (1 + n + i1) mod 2 = 0 and -n < i1 < n }")

In [59]:
# codegen(S * D)
D = idom
print("Schedule: ", S)
print("Iteration Domain: ", D)
out_schedule0 = S.intersect_domain(D)
print(type(out_schedule0), out_schedule0)

Schedule:  { S2[i] -> [1, i]; S1[i] -> [0, i] }
Iteration Domain:  [n] -> { S1[i] : 0 <= i <= n; S2[i] : 0 <= i <= n }
<class 'islpy._isl.UnionMap'> [n] -> { S2[i] -> [1, i] : 0 <= i <= n; S1[i] -> [0, i] : 0 <= i <= n }


In [63]:
# These print utility functions I copied from Tobias Grosser's talk:
# https://www.youtube.com/watch?v=mIBUY20d8c8
def printAST(ast):
    p = Printer.to_str(ast.get_ctx())
    p = p.set_output_format(format.C)
    p = p.print_ast_node(ast)
    print(p.get_str())
    
def printSchedule(schedule):
    p = Printer.to_str(schedule.get_ctx())
    p = p.set_yaml_style(yaml_style.BLOCK)
    p = p.print_schedule(schedule)
    print(p.get_str())
    
def printNode(node):
    p = Printer.to_str(node.get_ctx())
    p = p.set_yaml_style(yaml_style.BLOCK)
    p = p.print_schedule_node(node)
    print(p.get_str())
    
def p(obj):
    if (obj.__class__ == Schedule):
        printSchedule(obj)
    if (obj.__class__ == ScheduleNode):
        printNode(obj)
    if (obj.__class__ == AstNode):
        printAST(obj)

def printC(schedule):
    astbuild = AstBuild.from_context(Set("{:}"))
    ast = astbuild.node_from_schedule(schedule)
    p(ast)
    
def printCMap(schedule):
    astbuild = AstBuild.from_context(Set("{:}"))
    ast = astbuild.node_from_schedule_map(schedule)
    p(ast)

In [64]:
printCMap(out_schedule0)

{
  for (int c1 = 0; c1 <= n; c1 += 1)
    S1(c1);
  for (int c1 = 0; c1 <= n; c1 += 1)
    S2(c1);
}



In [65]:
S2 = UnionMap("[N]->{S1[i]->[i,0];S2[i]->[N-i-1,1]}") #schedule - fuse S1 and S2
out_schedule1 = S2.intersect_domain(D)
printCMap(out_schedule1)

{
  for (int c0 = -n + N - 1; c0 < min(0, N); c0 += 1)
    S2(N - c0 - 1);
  for (int c0 = 0; c0 <= n; c0 += 1) {
    S1(c0);
    if (N >= c0 + 1 && n + c0 + 1 >= N)
      S2(N - c0 - 1);
  }
  for (int c0 = max(n + 1, -n + N - 1); c0 < N; c0 += 1)
    S2(N - c0 - 1);
}

